In [909]:
#Importing Libraries
import pandas as pd 
from datetime import datetime
import xlwt 
from xlwt import Workbook 

In [910]:
#Reading in Datasets
cgmDF = pd.read_csv('CGMData.csv')
insDF = pd.read_csv('InsulinData.csv')

In [911]:
#1) Determing the time stamp when Auto mode starts
for i in range(0, len(insDF)):
    if(insDF.loc[i, "Alarm"] == 'AUTO MODE ACTIVE PLGM OFF'):
        iAutoDate = insDF.loc[i, "Date"]
        iAutoTime = insDF.loc[i, "Time"]

In [912]:
#2) Determining timestamp in CGMData.csv where Auto mode starts
import datetime

for i in range(0,len(cgmDF)):
    if(cgmDF.loc[(len(cgmDF) - i-1),"Date"] >= iAutoDate and cgmDF.loc[(len(cgmDF) - i-1), "Time"] >= iAutoTime):
        cAutoDateString = cgmDF.loc[(len(cgmDF) - i-1), "Date"]
        cAutoTimeString = cgmDF.loc[(len(cgmDF) - i-1), "Time"]
        break;

#Converting autodate and time from string to daytime object
cAutoDate = datetime.datetime.strptime(cAutoDateString, '%m/%d/%Y')
cAutoTime = datetime.datetime.strptime(cAutoTimeString, '%H:%M:%S')

print(cAutoDate)
print(cAutoTime)

2017-08-09 00:00:00
1900-01-01 08:10:05


In [913]:
#3) Converting string columns to date time objects
cgmDF['Date'] = pd.to_datetime(cgmDF['Date'], format="%m/%d/%Y")
cgmDF['Time'] = pd.to_datetime(cgmDF['Time'], format="%H:%M:%S")

type(cgmDF['Date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [914]:
#Creating time variables
import datetime
midnightString = "0:00:00"
midnight = datetime.datetime.strptime(midnightString, '%H:%M:%S')

sixAMString = "6:00:00"
sixAM = datetime.datetime.strptime(sixAMString, '%H:%M:%S')

In [915]:
#4) Parsing CGM Data by Whole Day, Overnight, and Daytime
#Creating lists that will store first row of every new day, start and end of daytime and overnight
days = []
overnight = []
daytime = []

lastDay = cgmDF.loc[0,"Date"]
lastTime = cgmDF.loc[0,"Time"]

#Appending first day (last row of dataset) to appropriate list(s)
if(cgmDF.loc[len(cgmDF) - 1, "Time"] >= midnight and cgmDF.loc[len(cgmDF) - 1, "Time"] < sixAM):
    overnight.append(len(cgmDF)+1)
    days.append(len(cgmDF) + 1)
elif(cgmDF.loc[len(cgmDF) - 1, "Time"] >= sixAM):
    daytime.append(len(cgmDF) + 1)
    days.append(len(cgmDF) + 1)

#Appending first row of each new day to days list
for i in range(1, len(cgmDF)):
    currentDay = len(cgmDF) - i
    currentTime = len(cgmDF) - i
    
    if(currentDay == lastDay and currentTime == lastTime): #Prevents index range error
        break;
    #Parsing Whole Day
    elif(cgmDF.loc[(len(cgmDF) - i), "Time"] < cgmDF.loc[(len(cgmDF) - i - 1), "Time"] ):
        pass
    else:
        days.append(len(cgmDF) - i+2) #days.append(len(cgmDF) - i+1)
        overnight.append(len(cgmDF) - i+1)
        daytime.append(len(cgmDF) - i+2)
        
    #Parsing Overnight and Daytime Subintervals
    #Looking for end of overnight and start of daytime
    if(cgmDF.loc[(len(cgmDF) - i), "Time"] < sixAM  and cgmDF.loc[(len(cgmDF) - i-1), "Time"] > sixAM):
        overnight.append(len(cgmDF) - i+2)
        daytime.append(len(cgmDF) - i+1)

#Apending last day (first row of dataset) to appropriate list(s)
firstRow = 0
if(cgmDF.loc[firstRow, "Time"] > sixAM):
    daytime.append(firstRow)
    days.append(firstRow)
elif(cgmDF.loc[firstRow, "Time"] <= sixAM):
    overnight.append(firstRow)
    days.append(firstRow)

In [916]:
#5) Function to put metrics into Results.csv
import csv

def insertMetricsAuto(aResultA, bResultA, cResultA, dResultA, eResultA, fResultA, segment):
    #Inserting metrics into Auto row in Results.csv
    if(segment == overnight):
        sheet1.write(2, 1, aResultA) 
        sheet1.write(2, 2, bResultA)
        sheet1.write(2, 3, cResultA)
        sheet1.write(2, 4, dResultA)
        sheet1.write(2, 5, eResultA)
        sheet1.write(2, 6, fResultA)
    elif(segment == daytime):
        sheet1.write(2, 7, aResultA) 
        sheet1.write(2, 8, bResultA)
        sheet1.write(2, 9, cResultA)
        sheet1.write(2, 10, dResultA)
        sheet1.write(2, 11, eResultA)
        sheet1.write(2, 12, fResultA)
    elif(segment == days):
        sheet1.write(2, 13, aResultA) 
        sheet1.write(2, 14, bResultA)
        sheet1.write(2, 15, cResultA)
        sheet1.write(2, 16, dResultA)
        sheet1.write(2, 17, eResultA)
        sheet1.write(2, 18, fResultA)
    
def insertMetricsManual(aResultM, bResultM, cResultM, dResultM, eResultM, fResultM, segment):
    #Inserting metrics into Manual row in Results.csv
    if(segment == overnight):
        sheet1.write(1, 1, aResultM) 
        sheet1.write(1, 2, bResultM)
        sheet1.write(1, 3, cResultM)
        sheet1.write(1, 4, dResultM)
        sheet1.write(1, 5, eResultM)
        sheet1.write(1, 6, fResultM)
    elif(segment == daytime):
        sheet1.write(1, 7, aResultM) 
        sheet1.write(1, 8, bResultM)
        sheet1.write(1, 9, cResultM)
        sheet1.write(1, 10, dResultM)
        sheet1.write(1, 11, eResultM)
        sheet1.write(1, 12, fResultM)
    elif(segment == days):
        sheet1.write(1, 13, aResultM) 
        sheet1.write(1, 14, bResultM)
        sheet1.write(1, 15, cResultM)
        sheet1.write(1, 16, dResultM)
        sheet1.write(1, 17, eResultM)
        sheet1.write(1, 18, fResultM)

In [917]:
#Getting rid of null values and replacing with 0
df = cgmDF["Sensor Glucose (mg/dL)"].isna()
for i in range(0, len(df)):
    if(df[i] == True): #If value is null replace with 0, otherwise do nothing
        cgmDF.loc[i, "Sensor Glucose (mg/dL)"] = 0

In [918]:
#Making sensor glucose column an integer type
cgmDF["Sensor Glucose (mg/dL)"] = cgmDF["Sensor Glucose (mg/dL)"].astype(int)

In [919]:
#Creating spreadsheet
wb = Workbook() 

#Writing column names:
sheet1 = wb.add_sheet('Metrics Sheet 2', cell_overwrite_ok=True) 
sheet1.write(0, 0, "Mode")
sheet1.write(1,0, "Manual Mode")
sheet1.write(2,0, "Auto Mode")

c=1
for i in range(0,6):
    sheet1.write(0,c,"overnight")
    c = c+1

for i in range(0,6):
    sheet1.write(0,c,"daytime")
    c = c+1
    
for i in range(0,6):
    sheet1.write(0,c,"whole day")
    c = c+1

In [920]:
#6) Function that calculates metrics for a specific segment
def gatherMetrics(segmentList):
    #Instantiating variables
    a = 0
    b = 0
    c = 0
    d = 0
    e = 0
    f = 0

    aTotal = 0
    bTotal = 0
    cTotal = 0
    dTotal = 0
    eTotal = 0
    fTotal = 0

    length = 0
    switched = False 

    for j in range(0, len(segmentList)-1):
        intervalLen = segmentList[j] - segmentList[j+1]
        i = 0
        for i in range(segmentList[j], segmentList[j+1], -1):
            if(i >= 2):
                #Collecting data for metric computation
                if(cgmDF.loc[i-2, "Sensor Glucose (mg/dL)"] == 0): #If null value, don't increment anything but reduce size of interval
                     intervalLen = intervalLen - 1
                elif(cgmDF.loc[i-2, "Sensor Glucose (mg/dL)"] > 250):
                    b = b +1
                elif(cgmDF.loc[i-2, "Sensor Glucose (mg/dL)"] > 180):
                    a  = a +1
                elif(cgmDF.loc[i-2, "Sensor Glucose (mg/dL)"] >= 70 and cgmDF.loc[i-2, "Sensor Glucose (mg/dL)"] <= 150):
                    d  = d +1
                elif(cgmDF.loc[i-2, "Sensor Glucose (mg/dL)"] >= 70 and cgmDF.loc[i-2, "Sensor Glucose (mg/dL)"] <= 180):
                    c  = c +1
                elif(cgmDF.loc[i-2, "Sensor Glucose (mg/dL)"] < 54):
                    f  = f +1
                elif(cgmDF.loc[i-2, "Sensor Glucose (mg/dL)"] < 70):
                    e  = e +1

                #Getting final averages to put into manual mode if we just switched to auto modeResults.csv
                if(cgmDF.loc[i-2, "Date"] >= cAutoDate and switched == False):
                    if(cgmDF.loc[i-2, "Time"] >= cAutoTime):
                        aResultM = aTotal / length 
                        bResultM = bTotal / length 
                        cResultM = cTotal / length 
                        dResultM = dTotal / length 
                        eResultM = eTotal / length 
                        fResultM = fTotal / length 
                        switched = True
                        
                        #Inserting metrics to Manual Mode row in Results.csv
                        insertMetricsManual(aResultM, bResultM,cResultM,dResultM,eResultM,fResultM, segmentList)
                        
                        #Resetting variables
                        a = 0
                        b = 0
                        c = 0
                        d = 0
                        e = 0
                        f = 0
                        aTotal = 0
                        bTotal = 0
                        cTotal = 0
                        dTotal = 0
                        eTotal = 0
                        fTotal = 0
                        length = 0
                        intervalLen = (segmentList[j] - i) - segmentList[j+1] #Resetting interval to start at auto mode

        #Calculating metric percentages
        aP = a / intervalLen
        bP = b / intervalLen
        cP = c / intervalLen
        dP = d / intervalLen
        eP = e / intervalLen
        fP = f / intervalLen

        #Adding percentage to current total
        aTotal = aTotal + aP
        bTotal = bTotal + bP
        cTotal = cTotal + cP
        dTotal = dTotal + dP
        eTotal = eTotal + eP
        fTotal = fTotal + fP
        length = length + 1   

        #Resetting variables to 0
        a = 0
        b = 0
        c = 0
        d = 0
        e = 0
        f = 0

    #Inserting metrics to Auto Mode row in Results.csv 
    aResultA = aTotal / length 
    bResultA = bTotal / length 
    cResultA = cTotal / length 
    dResultA = dTotal / length 
    eResultA = eTotal / length 
    fResultA = fTotal / length 
    
    #Write code on line below to insert into results.csv
    insertMetricsAuto(aResultA,bResultA,cResultA,dResultA,eResultA,fResultA, segmentList)

In [921]:
#7) Gathering metrics for Overnight interval and placing in Results.csv
gatherMetrics(overnight)   

In [922]:
#8) Gathering metrics for Daytime interval and inserting into Results.csv
gatherMetrics(daytime)

In [923]:
#9) Gathering metrics for Whole Day interval and inserting into Results.csv
gatherMetrics(days)

In [924]:
#10) Saving Spreadsheet
wb.save('Steinberg_Results.csv') 
print("Finished Compiling!")

Finished Compiling!
